In [ ]:

# Read SV file
df = pd.read_csv("/scratch/tweber/SCO_COURSE/HJ_MIXTURE_RPE1_Mix/RPE1_Mix/mosaiclassifier/sv_calls/stringent_filterTRUE.tsv", sep="\t")
# df = pd.read_csv("../stringent_filterTRUE.tsv", sep="\t")
df["ID"] = df["chrom"] + "_" + df["start"].astype(str) + "_" + df["end"].astype(str)

# Read 200kb bins file
binbed = pd.read_csv(
    # "../bin_200kb_all.bed",
    "workflow/data/bin_200kb_all.bed",
    sep="\t",
    names=["chrom", "start", "end", "bin_id"],
)
binbed["ID"] = binbed["chrom"] + "_" + binbed["start"].astype(str) + "_" + binbed["end"].astype(str)

# Turn chrom into categorical
binbed["chrom"] = pd.Categorical(
    binbed["chrom"],
    categories=["chr{}".format(e) for e in range(1, 23)] + ["chrX", "chrY"],
    ordered=True,
)

# Sort & filter out chrY #TMP / can be changed
binbed = binbed.sort_values(by=["chrom", "start", "end"]).reset_index(drop=True)
binbed = binbed.loc[~binbed["chrom"].isin(["chrY"])]

# Instanciate final list
l = list()


def process_row(r):
    """Get all bins from binbed that overlap SV call

    Args:
        r (pandas row)
    """
    tmp_r = binbed.loc[(binbed["chrom"] == r["chrom"]) & (binbed["start"] >= r["start"]) & (binbed["end"] <= r["end"])]
    tmp_r["cell"] = r["cell"]
    tmp_r["sv_call_name"] = r["sv_call_name"]
    tmp_r["af"] = r["af"]
    tmp_r["llr_to_ref"] = r["llr_to_ref"]
    # Append result to list
    l.append(tmp_r)


# Apply & loop on each temporary cell dataframe created
def process_sv(tmp_df):
    tmp_df.apply(lambda r: process_row(r), axis=1)


# Create a nested pandas apply
df.groupby("cell").apply(lambda r: process_sv(r))

# Concat results
processed_df = pd.concat(l)
processed_df["ID"] = processed_df["chrom"].astype(str) + "_" + processed_df["start"].astype(str) + "_" + processed_df["end"].astype(str)

# Extract only empty bins (outer join) from binbed
binbed_not_used = binbed.loc[~binbed["ID"].isin(processed_df.ID.unique().tolist())]
# Concat with previously created dataframe
concat_df = pd.concat([processed_df, binbed_not_used])

In [1]:
import pandas as pd

# Read 200kb bins file
binbed = pd.read_csv(
    # "../bin_200kb_all.bed",
    "workflow/data/bin_200kb_all.bed",
    sep="\t",
    names=["chrom", "start", "end", "bin_id"],
)
binbed["ID"] = binbed["chrom"] + "_" + binbed["start"].astype(str) + "_" + binbed["end"].astype(str)
binbed



FileNotFoundError: [Errno 2] No such file or directory: 'workflow/data/bin_200kb_all.bed'